# Installing & Imports

#Functions we have so far: 
- Exclusive bad (Katie - delete the words that do not make sense and finish up  )
- Exclusive good (Katie - delete the words that do not make sense and finish up  )
- Exclamation mark (Debbie)
- Question mark (Debbie)
- Capital letters (Debbie)
- too (negative) (Vera)
- far (positive) (Vera)
- n\'t (Hong)
- positive word (Katie)
- negative word (Katie)

In [1]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-0.24.2              |   py36he6710b0_0         8.5 MB
    ------------------------------------------------------------
                                           Total:         8.5 MB

The following packages will be DOWNGRADED:

  pandas                               1.0.1-py36h0573a6f_0 --> 0.24.2-py36he6710b0_0



pandas-0.24.2        | 8.5 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda upgrade --all -y

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.3.2           |   py36heb32a55_0         124 KB
    dask-2.11.0                |             py_0          13 KB
    dask-core-2.11.0           |             py_0         565 KB
    distributed-2.11.0         |           py36_0         943 KB
    gunicorn-20.0.4            |           py36_0         125 KB
    hypothesis-5.5.4           |             py_0         227 KB
    nb_conda_kernels-2.2.2     |           py36_0          39 KB
    ncurses-6.2                |       he6710b0_0         1.1 MB
    plotly-4.5.2               |             py_0         1.6 MB
    pytest-astropy-0.8.0       |             py_0           9 KB
    ------------------------------------------------------------
                                         

In [3]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    cudatoolkit-10.0.130       |                0       261.2 MB
    cudnn-7.6.5                |       cuda10.0_0       165.0 MB
    libprotobuf-3.11.3         |       h8b12597_0         4.8 MB  conda-forge
    ninja-1.10.0               |       hc9558a2_0         1.9 MB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    pandas-0.24.2              |   py36hb3f55d8_1        11.1 MB  conda-forge
    protobuf-3.11.3       

In [4]:
conda install -c conda-forge textblob

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - textblob


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    textblob-0.15.3            |             py_0         595 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         595 KB

The following NEW packages will be INSTALLED:

  textblob           conda-forge/noarch::textblob-0.15.3-py_0



textblob-0.15.3      | 595 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [5]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import nltk
from itertools import repeat
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests
from bs4 import BeautifulSoup
from csv import writer
import re
import pickle
import time
import json

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/faculty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#Spark 

# Spark Environment
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import pyspark

number_cores = 4
memory_gb = 16
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext.getOrCreate(conf=conf)
print(sc)

# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

<SparkContext master=local[4] appName=pyspark-shell>


# Web Scraping

## Browse all from DVD releases page

In [7]:
main = 'https://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&certified&sortBy=release&type=dvd-streaming-all&page='

In [8]:
# Get movie url
movie_url = []
start_page = 1 ; end_page = 1
while start_page <= end_page:
#     time.sleep(7)
    url = main + str(start_page)
    response = requests.get(url)
    if response.status_code !=200:
        print('Request error')
        break
    file = json.loads(response.text)
    for i in file['results']:
        movie_url = movie_url + [i['url']]
    start_page +=1

In [9]:
print('Examples for the url:\n')
for i in range(3):
    print(movie_url[i])
print('\nNumber of movies in list: {}'.format(len(movie_url)))

Examples for the url:

/m/frozen_ii
/m/playmobil_the_movie
/m/queen_and_slim

Number of movies in list: 32


In [10]:
# Split into lists of 50 movies to do the scraping
movie_url_split = [movie_url[i:i+50] for i in range(0,600,50)]

In [11]:
len(movie_url_split)

12

In [12]:
# Get reviews from the web
reviews = []
titles = []
ratings = []
for split in movie_url_split: # Loop through each split
#     time.sleep(7)
    for title in split: # Loop through each movie title
        url = 'https://www.rottentomatoes.com'+title
#         time.sleep(7)
        response = requests.get(url)
        # Check the request status code
        if response.status_code != 200:
            print('Request error')
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get labels from each review (fresh vs. rotten)
        fresh_rotten = soup.find_all(class_="review_quote")
        
        # Get movie title
        title = soup.find(class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").getText()
        
        # Get reviews
        review = soup.find_all('blockquote')
        for i in review:
            j = str(i.contents[1])
            j = j.replace("<p>\n                    \n                        ","")
            j = j.replace("\n                    \n                </p>","")
            reviews = reviews + [j]
            titles = titles + [title]
        
        # Identify labels
        for i in fresh_rotten:
            temp = str(i.findChildren()[2])
            if re.search('rotten',temp):
                ratings = ratings + ['rotten']
            else:
                ratings = ratings + ['fresh']
            

In [13]:
# Create the DataFrame to store the scraped data
df = pd.DataFrame([titles,reviews,ratings],index = ['title','review','rating']).T

In [14]:
# Clean the data (drop duplicates, check missing values etc.)
df = df.drop_duplicates()
df = df.replace([None],np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 330
Data columns (total 3 columns):
title     330 non-null object
review    330 non-null object
rating    330 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


In [15]:
df.dropna(inplace=True)
df.head(3)

,title,review,rating
0,Frozen II,A distressingly unnecessary (and fairly tediou...,rotten
1,Frozen II,Frozen II is the exact kind of sequel that mor...,fresh
2,Frozen II,[A]n ultimately satisfying story of more matur...,fresh


In [16]:
# Export to CSV files
# df.to_csv('web_scraping_rotten_tomatoes.csv')

# Reading & Preparing TSV file 

In [17]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [18]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [19]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [20]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [21]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [22]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [23]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [24]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [25]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews = tsv_reviews.sample(5000)
#take 5000
tsv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 8753 to 19142
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null object
dtypes: object(2)
memory usage: 117.2+ KB


# Reading & Preparing CSV file 

In [26]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [27]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)
csv_reviews = csv_reviews.sample(5000)

csv_reviews.head()
csv_reviews.info()

,Review,Freshness
137829,Lives spent looking at devices and screens is...,1
307429,An engaging blend of modern-day interviews an...,1
40117,"A difficult film to like, an impossible one n...",1
463932,American Ninja was still a compltely awful mo...,0
382015,"Fame 2009 has turns rather than scenes, and i...",0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 137829 to 393444
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null int64
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


# Web Scraping & Preparing scrapped data 

In [28]:
#Read Web Scraping Data
web_scraping_reviews= pd.read_csv('/project/web_scraping_rotten_tomatoes.csv')
web_scraping_reviews.head()

web_scraping_reviews = web_scraping_reviews.sample(5000)

,Unnamed: 0,title,review,rating
0,0,My Hindu Friend (Meu amigo Hindu),Babenco's cinematic farewell isn't perfect by ...,fresh
1,1,My Hindu Friend (Meu amigo Hindu),This is a good film if you are looking for som...,fresh
2,2,My Hindu Friend (Meu amigo Hindu),"My Hindu Friend is a celebration of life, love...",fresh
3,3,My Hindu Friend (Meu amigo Hindu),I wouldn't miss it; it's a film that's more th...,fresh
4,4,My Hindu Friend (Meu amigo Hindu),"...surreal, reflective (though never sentiment...",fresh


In [29]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews.head()

# Extract Review and Freshness columns
web_scraping_reviews= pd.DataFrame(web_scraping_reviews, columns = ['Review', 'Freshness'])

,Unnamed: 0,title,Review,Freshness
2295,2295,The Day Shall Come,"It's a case of distance guiding the film, with...",0
3529,3529,Charlie Says,"Charlie Says, for all its contemplative trappi...",1
5194,5194,Avengement,"The ""New Jason Statham"" Adkins is still great ...",0
4484,4484,Us,Us revolves around an allegory that's more ela...,1
1471,1471,The Return of Martin Guerre (Le Retour de Mart...,A sobering account not only Medieval provincia...,1


In [30]:
web_scraping_reviews.head()

,Review,Freshness
2295,"It's a case of distance guiding the film, with...",0
3529,"Charlie Says, for all its contemplative trappi...",1
5194,"The ""New Jason Statham"" Adkins is still great ...",0
4484,Us revolves around an allegory that's more ela...,1
1471,A sobering account not only Medieval provincia...,1


# Combining all the data together 

In [31]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
137829,Lives spent looking at devices and screens is...,1
307429,An engaging blend of modern-day interviews an...,1
40117,"A difficult film to like, an impossible one n...",1
463932,American Ninja was still a compltely awful mo...,0
382015,"Fame 2009 has turns rather than scenes, and i...",0


In [32]:
all_reviews.shape

(15000, 2)

# Split into test and training set 

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
all_reviews['Freshness'] = all_reviews['Freshness'].astype(int)

In [35]:
train, test = train_test_split(all_reviews,test_size=0.2,stratify = all_reviews['Freshness'])

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 4195 to 43860
Data columns (total 2 columns):
Review       12000 non-null object
Freshness    12000 non-null int64
dtypes: int64(1), object(1)
memory usage: 281.2+ KB


In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 2360 to 1116
Data columns (total 2 columns):
Review       3000 non-null object
Freshness    3000 non-null int64
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [38]:
train.head()

,Review,Freshness
4195,A sobering portrait of human trafficking for s...,1
4495,"Solid hip hop genre picture that gives ""Black-...",1
92900,"Superbly directed, thought-provoking blend of...",1
28943,Dumb comedy should be cited for bad taste.,0
2455,Before an inexpressible and unreachable realit...,1


In [39]:
#get rid off training labels 
train = train.drop('Freshness', 1)

In [40]:
test['Freshness'].value_counts()

1    1758
0    1242
Name: Freshness, dtype: int64

In [41]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(2000,random_state=42)
development_split.head()


,Review,Freshness
52316,"With its distinctive voice, the irreverent com...",1
1195,A shameless crowd-pleaser delivered with such ...,1
94817,Those who love it as passionately as I do rea...,1
2883,There's great satisfaction and hope in Darlin'...,1
3323,A first date gone bad...then good...then bad.....,1


In [42]:
#For finding labelling functions: 
development_split

,Review,Freshness
52316,"With its distinctive voice, the irreverent com...",1
1195,A shameless crowd-pleaser delivered with such ...,1
94817,Those who love it as passionately as I do rea...,1
2883,There's great satisfaction and hope in Darlin'...,1
3323,A first date gone bad...then good...then bad.....,1
5143,The way every ruffle of a dress or lace embell...,1
673,This is one of those movies that you have to w...,1
467842,"A very stupid, very nasty and very messy horr...",0
469998,The sharp comic timing and devil-may-care bre...,0
409,"From his time in the wilderness, Almodóvar has...",1


In [43]:
development_split.count()

Review       2000
Freshness    2000
dtype: int64

In [44]:
development_split.to_csv('development_split.csv')

In [45]:
development_split.count()

Review       2000
Freshness    2000
dtype: int64

In [46]:
development_split.head()

,Review,Freshness
52316,"With its distinctive voice, the irreverent com...",1
1195,A shameless crowd-pleaser delivered with such ...,1
94817,Those who love it as passionately as I do rea...,1
2883,There's great satisfaction and hope in Darlin'...,1
3323,A first date gone bad...then good...then bad.....,1


In [47]:
#Might have to get rid off index?

development_split[development_split['Freshness'] !=1].count()

Review       843
Freshness    843
dtype: int64

In [48]:
development_split['Freshness'].value_counts()

1    1157
0     843
Name: Freshness, dtype: int64

## Split into positive and negative reviews

In [49]:
development_split_fresh = development_split[development_split['Freshness'] == 1]
development_split_rotten = development_split[development_split['Freshness'] == 0]
development_split_fresh.head()
development_split_rotten.head()

,Review,Freshness
52316,"With its distinctive voice, the irreverent com...",1
1195,A shameless crowd-pleaser delivered with such ...,1
94817,Those who love it as passionately as I do rea...,1
2883,There's great satisfaction and hope in Darlin'...,1
3323,A first date gone bad...then good...then bad.....,1


,Review,Freshness
467842,"A very stupid, very nasty and very messy horr...",0
469998,The sharp comic timing and devil-may-care bre...,0
3672,Even undemanding kids in need of distraction m...,0
21320,"Once things get going, you might wonder why th...",0
4048,The world is fantastically realized. From the ...,0


In [50]:
# fresh reviews 
development_split_fresh.info()
# rotten reviews 
development_split_rotten.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1157 entries, 52316 to 36181
Data columns (total 2 columns):
Review       1157 non-null object
Freshness    1157 non-null int64
dtypes: int64(1), object(1)
memory usage: 27.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 843 entries, 467842 to 297367
Data columns (total 2 columns):
Review       843 non-null object
Freshness    843 non-null int64
dtypes: int64(1), object(1)
memory usage: 19.8+ KB


## Labelling Functions

## 1. Word occurrences

### Positive Reviews

In [51]:
# Removing punctuation 
def remove_punctuation(dataframe):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in dataframe.Review.str.lower():
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [52]:
#Remove punctuation from fresh reviews & turn into Series
split_fresh= pd.Series(remove_punctuation(development_split_fresh))
split_fresh.head()

0    with its distinctive voice the irreverent come...
1    a shameless crowdpleaser delivered with such p...
2     those who love it as passionately as i do rea...
3    theres great satisfaction and hope in darlin b...
4    a first date gone badthen goodthen badthen goo...
dtype: object

In [53]:
stopWordList = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves",\
                "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",\
                "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was",\
                "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and",\
                "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between",\
                "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",\
                "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any",\
                "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",\
                "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [54]:
#Removing stopwords from fresh

replacements = dict(zip((fr'\b{word}\b' for word in stopWordList), repeat("")))
split_fresh.replace(replacements, regex=True, inplace=True)
split_fresh.replace({r' +': ' ', r' +\.': '.'}, regex=True, inplace=True)

### Implement lemmatization

In [55]:
# Lemmatization and Count again
def stem_recount(df):
    import pandas as pd
    # Lemmatization
    from nltk import LancasterStemmer
    st = LancasterStemmer()
    newdf = df.copy()
    for i in range(0,len(df)):
        newdf.iloc[i,0] = st.stem(str(df.iloc[i,0])) 
        # Plz make sure the word column is the first column in df when using this function
    
    # Recount
    duplicate = newdf[newdf.duplicated(['index'])]
    # Plz make sure the 'index' is the column name consisting of words
    for i in range(0,len(newdf)):
        if i >= len(duplicate):
            break
        if newdf.iloc[i,0] == duplicate.iloc[i,0]:
            newdf.iloc[i,1] = newdf.iloc[i,1] + duplicate.iloc[i,1]
    return newdf

In [56]:
common_words_fresh = split_fresh.str.split(expand=True).stack().value_counts()
common_words_fresh_df = pd.DataFrame(common_words_fresh)
common_words_fresh_df = common_words_fresh_df.rename({0:'Occurence good review'}, axis='columns')
new_common_words_fresh_df = common_words_fresh_df.reset_index()

In [57]:
stem_recount(new_common_words_fresh_df)

,index,Occurence good review
0,film,240
1,movy,110
2,on,105
3,lik,72
4,story,64
5,ful,55
6,ev,52
7,span,48
8,review,47
9,good,45


In [58]:
#Get most common words in positive reviews 
top_common_words_fresh = common_words_fresh_df[common_words_fresh_df['Occurence good review'] >=4]
top_common_words_fresh

,Occurence good review
film,199
movie,110
one,105
like,72
story,64
full,55
even,52
spanish,48
review,47
good,45


### *** EXPLAIN WHY WE DONT USE LEMMATIZATION

<div class="alert alert-success">
<b> Reason for not using Lemmatization </b>

<p>
    Before counting the occurance of words in the movie review, we noticed that inflections in words may result in different occurances and thus generating bias during counting. For example, "enjoy" and "enjoyed" share the same root but would be counted separately if not using Lemmatization.
    </p> 
    
<p>
    The function "stem_recount" takes the root of a word and recounts the occurences. However, it posed a disadvantage of mis-normalizing words into other completely different words. For example, "movie" was identified as "movy", and "like" was identified as "lik". We thought this disadvantage exceeds the benefits of correcting word inflection, so we decided to not implement it.
    </p> 

</div>

### Negative reviews

In [59]:
#Remove punctuation from rotten & turn into Series
split_rotten= pd.Series(remove_punctuation(development_split_rotten))
split_rotten.head()

0     a very stupid very nasty and very messy horro...
1     the sharp comic timing and devilmaycare breez...
2    even undemanding kids in need of distraction m...
3    once things get going you might wonder why the...
4    the world is fantastically realized from the c...
dtype: object

In [60]:
# Removing stopwords from negative reviews
replacements = dict(zip((fr'\b{word}\b' for word in stopWordList), repeat("")))
split_rotten.replace(replacements, regex=True, inplace=True)
split_rotten.replace({r' +': ' ', r' +\.': '.'}, regex=True, inplace=True)

In [61]:
#Get most common words in negative reviews 

common_words_rotten = split_rotten.str.split(expand=True).stack().value_counts()
common_words_rotten_df = pd.DataFrame(common_words_rotten)
common_words_rotten_df = common_words_rotten_df.rename({0:'Occurence bad review'}, axis='columns')
top_common_words_rotten = common_words_rotten_df[common_words_rotten_df['Occurence bad review'] >=4]

In [62]:
top_common_words_rotten.head()

,Occurence bad review
film,119
movie,103
like,74
one,53
good,47


## Comparison of good and bad reviews

<div class="alert alert-success">
We want to find out which of the words in the good list only appear in the good movies (and not in the bad movies), vice versa and base labeling functions on these findings. We first ened to prepare the data accordingly, before we can write the labelling functions.
    <div>

In [63]:
top_fresh_words_exclusive = top_common_words_fresh.merge(top_common_words_rotten, indicator='i', how='outer', left_index=True,\
                                                         right_index=True).query('i == "left_only"').drop('i', 1)

top_rotten_words_exclusive = top_common_words_rotten.merge(top_common_words_fresh, indicator='i', how='outer', left_index=True,\
                                                           right_index=True).query('i == "left_only"').drop('i', 1)

In [64]:
top_fresh_words_exclusive.head()

,Occurence good review,Occurence bad review
2019,5.0,NaN
able,5.0,NaN
absolutely,4.0,NaN
absorbing,7.0,NaN
actions,5.0,NaN


In [65]:
top_rotten_words_exclusive.head()

,Occurence bad review,Occurence good review
across,4.0,NaN
already,6.0,NaN
amusing,4.0,NaN
anyone,5.0,NaN
awful,4.0,NaN


In [66]:
#Get only positive words 
top_fresh_words_exclusive_list = top_fresh_words_exclusive['Occurence good review'].index.tolist()
top_fresh_words_exclusive_list

['2019',
 'able',
 'absolutely',
 'absorbing',
 'actions',
 'actor',
 'actress',
 'adaptation',
 'age',
 'aging',
 'air',
 'alien',
 'although',
 'always',
 'amount',
 'amp',
 'angry',
 'animation',
 'apart',
 'art',
 'artist',
 'attention',
 'aussie',
 'authentic',
 'away',
 'baby',
 'baseball',
 'beats',
 'beautifully',
 'beauty',
 'become',
 'becomes',
 'behind',
 'believe',
 'bittersweet',
 'black',
 'bleak',
 'blockbuster',
 'bloody',
 'blue',
 'bmovie',
 'body',
 'books',
 'boys',
 'breath',
 'bright',
 'brilliant',
 'brutal',
 'camera',
 'captivating',
 'captures',
 'center',
 'certain',
 'certainly',
 'changes',
 'chemistry',
 'childhood',
 'children',
 'chilling',
 'cinema',
 'cinematography',
 'class',
 'clearly',
 'cold',
 'color',
 'commercial',
 'commitment',
 'complex',
 'concerned',
 'consistently',
 'contemporary',
 'content',
 'cool',
 'core',
 'country',
 'crafted',
 'created',
 'creative',
 'credits',
 'cultural',
 'culture',
 'dance',
 'debut',
 'decades',
 'definit

In [67]:
#take out the ones that seem to make sense: 
top_fresh_words_exclusive = ['absolutely',
 'addition',
 'adventure',
 'affectionate',
 'amazing',
 'ambition',
 'art',
 'artist',
 'arts',
 'atmosphere',
 'attractive',
 'awards',
 'balance',
 'beautiful',
 'beautifully',
 'beauty',
 'bond',
 'bright',
 'brilliant',
 'captivating',
 'captures',
 'celebration',
 'charm',
 'charming',
 'christmas',
 'classic',
 'clever',
 'committed',
 'consistently',
 'contemporary',
 'conventional',
 'convincingly',
 'creates',
 'creating',
 'crowdpleaser',
 'cult',
 'decade',
 'decades',
 'deep',
 'deeper',
 'deeply',
 'definitely',
 'delightful',
 'delightfully',
 'depth',
 'deserves',
 'design',
 'details',
 'different',
 'diverse',
 'dramatic',
 'early',
 'elegant',
 'emotionally',
 'engaging',
 'enjoyable',
 'enjoyed',
 'equal',
 'especially',
 'exploration',
 'extraordinary',
 'extremely',
 'familiar',
 'famous',
 'fan',
 'fantastic',
 'fantasy',
 'fascinating',
 'felt',
 'filled',
 'finest',
 'frank',
 'fresh',
 'friends',
 'friendship',
 'gags',
 'gorgeous',
 'grand',
 'happy',
 'heart',
 'hilarious',
 'honest',
 'hope',
 'huge',
 'impact',
 'insightful',
 'inspiring',
 'intelligent',
 'intense',
 'intrigue',
 'joy',
 'laugh',
 'loved',
 'mature',
 'mind',
 'mystery',
 'nostalgia',
 'novel',
 'opening',
 'passion',
 'perfect',
 'performers',
 'personal',
 'pleasure',
 'poignant',
 'power',
 'powerful',
 'precisely',
 'profound',
 'project',
 'proves',
 'provide',
 'provocative',
 'psychological',
 'quality',
 'remarkable',
 'reveals',
 'rich',
 'riveting',
 'satisfying',
 'sharp',
 'simple',
 'smart',
 'smile',
 'stunning',
 'succeeds',
 'supernatural',
 'surprise',
 'surprises',
 'surprising',
 'surprisingly',
 'sweet',
 'talents',
 'thoughtful',
 'thrills',
 'touch',
 'touching',
 'tragedy',
 'tragic',
 'tribute',
 'unique',
 'universal',
 'warm',
 'watchable',
 'welcome',
 'wit',
 'witty',
 'wonderful',
 'worthwhile',
 'worthy']

top_fresh_words_exclusive

['absolutely',
 'addition',
 'adventure',
 'affectionate',
 'amazing',
 'ambition',
 'art',
 'artist',
 'arts',
 'atmosphere',
 'attractive',
 'awards',
 'balance',
 'beautiful',
 'beautifully',
 'beauty',
 'bond',
 'bright',
 'brilliant',
 'captivating',
 'captures',
 'celebration',
 'charm',
 'charming',
 'christmas',
 'classic',
 'clever',
 'committed',
 'consistently',
 'contemporary',
 'conventional',
 'convincingly',
 'creates',
 'creating',
 'crowdpleaser',
 'cult',
 'decade',
 'decades',
 'deep',
 'deeper',
 'deeply',
 'definitely',
 'delightful',
 'delightfully',
 'depth',
 'deserves',
 'design',
 'details',
 'different',
 'diverse',
 'dramatic',
 'early',
 'elegant',
 'emotionally',
 'engaging',
 'enjoyable',
 'enjoyed',
 'equal',
 'especially',
 'exploration',
 'extraordinary',
 'extremely',
 'familiar',
 'famous',
 'fan',
 'fantastic',
 'fantasy',
 'fascinating',
 'felt',
 'filled',
 'finest',
 'frank',
 'fresh',
 'friends',
 'friendship',
 'gags',
 'gorgeous',
 'grand',
 '

In [68]:
#Get only negative words 
top_rotten_words_exclusive_list = top_rotten_words_exclusive['Occurence bad review'].index.tolist()
top_rotten_words_exclusive_list

['across',
 'already',
 'amusing',
 'anyone',
 'awful',
 'bag',
 'banter',
 'basically',
 'battle',
 'beyond',
 'biggest',
 'bland',
 'boring',
 'brain',
 'british',
 'capable',
 'casting',
 'cheap',
 'child',
 'chris',
 'christopher',
 'clever',
 'completely',
 'contrived',
 'convincing',
 'couldnt',
 'couple',
 'course',
 'dead',
 'decent',
 'deserve',
 'deserves',
 'disappointment',
 'disaster',
 'dog',
 'edited',
 'else',
 'emotionally',
 'essentially',
 'example',
 'except',
 'expectations',
 'failure',
 'felt',
 'finally',
 'flat',
 'forget',
 'future',
 'halfbaked',
 'hannibal',
 'help',
 'historical',
 'home',
 'hot',
 'humour',
 'idea',
 'imagination',
 'impressive',
 'intentions',
 'interested',
 'involved',
 'jennifer',
 'jokes',
 'kill',
 'king',
 'lazy',
 'lessons',
 'lets',
 'limp',
 'lots',
 'low',
 'male',
 'mark',
 'means',
 'meant',
 'mediocre',
 'mediocrity',
 'middle',
 'misguided',
 'mistakes',
 'modest',
 'moment',
 'momentum',
 'money',
 'moving',
 'mr',
 'name',

In [69]:
#take out the ones that seem to make sense: 
top_rotten_words_exclusive = [
 'attempt',
 'awkward',
 'barely',
 'basically',
 'bizarre',
 'bland',
 'boring',
 'clumsy',
 'comedic',
 'disappointing',
 'disappointingly',
 'disappointment',
 'disaster',
 'dull',
 'effort',
 'failed',
 'fails',
 'generic',
 'irritating',
 'lacking',
 'manic',
 'missing',
 'nobody',
 'noir',
 'none',
 'painfully',
 'pointless',
 'poorly',
 'problem',
 'shallow',
 'shame',
 'sloppy',
 'slow',
 'suffers',
 'superficial',
 'try',
 'unfortunately',
 'unfunny',
 'worst']
top_rotten_words_exclusive

['attempt',
 'awkward',
 'barely',
 'basically',
 'bizarre',
 'bland',
 'boring',
 'clumsy',
 'comedic',
 'disappointing',
 'disappointingly',
 'disappointment',
 'disaster',
 'dull',
 'effort',
 'failed',
 'fails',
 'generic',
 'irritating',
 'lacking',
 'manic',
 'missing',
 'nobody',
 'noir',
 'none',
 'painfully',
 'pointless',
 'poorly',
 'problem',
 'shallow',
 'shame',
 'sloppy',
 'slow',
 'suffers',
 'superficial',
 'try',
 'unfortunately',
 'unfunny',
 'worst']

## Labelling Function

###  A. Good / bad exclusive words occurrences

In [120]:
development_split = pd.read_csv('/project/development_split.csv')
ABSTAIN = -1
NOTFRESH = 0
FRESH = 1

@labeling_function()
def fresh(x):
    for word in top_fresh_words_exclusive:
        if word in str(x).lower():
            return FRESH
    return ABSTAIN
#return FRESH if "best" in x.str.lower() else ABSTAIN

@labeling_function()
def rotten(x):
    for word in top_rotten_words_exclusive:
        if word in str(x).lower():
            return NOTFRESH
    return ABSTAIN
#return NOTFRESH if "best" in x.str.lower() else ABSTAIN

In [127]:
# lfs = [fresh]

# applier = PandasLFApplier(lfs=lfs)
# sample_L = applier.apply(development_split)


from snorkel.labeling.apply.spark import SparkLFApplier

from pyspark import SparkContext 
from pyspark.sql import SQLContext 
import pandas as pd 
sqlc=SQLContext(sc) 
df=pd.read_csv('/project/development_split.csv') 
development_split=sqlc.createDataFrame(df) 


In [134]:
lfs = [fresh]
applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [135]:
coverage_fresh = (sample_L != ABSTAIN).mean(axis=0)
print("fresh coverage:{:.1%}".format(coverage_fresh[0]))

fresh coverage:100.0%


In [136]:
lfs = [rotten]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [137]:
coverage_rotten = (sample_L != ABSTAIN).mean(axis=0)
print("rotten coverage:{:.1%}".format(coverage_rotten[0]))

rotten coverage:15.1%


### B. Word 'too' occurances

In [75]:
common_words_fresh_df[common_words_fresh_df.index == 'too']

,Occurence good review
too,29


In [76]:
common_words_rotten_df[common_words_rotten_df.index == 'too']

,Occurence bad review
too,29


In [138]:
@labeling_function()
def keyword_too(x):
    return NOTFRESH if 'too' in str(x).lower() else ABSTAIN

In [139]:
lfs = [keyword_too]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [140]:
coverage_keyword_too = (sample_L != ABSTAIN).mean(axis=0)
print("keyword too coverage:{:.1%}".format(coverage_keyword_too[0]))

keyword too coverage:3.9%


### C. Word 'far' occurrences

In [80]:
common_words_fresh_df[common_words_fresh_df.index == 'far']

,Occurence good review
far,18


In [81]:
common_words_rotten_df[common_words_rotten_df.index == 'far']

,Occurence bad review
far,11


In [141]:
@labeling_function()
def keyword_far(x):
    return FRESH if 'far' in str(x).lower() else ABSTAIN

In [142]:
lfs = [keyword_far]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [143]:
coverage_keyword_far = (sample_L != ABSTAIN).mean(axis=0)
print("keyword far coverage:{:.1%}".format(coverage_keyword_far[0]))

keyword far coverage:3.0%


### D. "n't" words occurrences

In [ ]:
@labeling_function()

def t(x):
    if re.search("'t",x.review.lower()):
        return NOTFRESH
    return ABSTAIN
#return FRESH if "best" in x.str.lower() else ABSTAIN

In [ ]:
lfs = [t]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [85]:
# Word occurancy that with punctuation with it

# Word occurrences dataframe for fresh reviews
development_split_fresh_1 = split_fresh.str.split(expand=True).stack().value_counts()
development_split_fresh_df = pd.DataFrame(development_split_fresh_1).reset_index()

# Words occurrences dataframe for rotten reviews
development_split_rotten_1 = split_rotten.str.split(expand=True).stack().value_counts()
development_split_rotten_df = pd.DataFrame(development_split_rotten_1).reset_index()

### E. Occurenes of good & bad words from external list 

<div class="alert alert-success">
We also want to look at an imported list of postive and negative words and see whether we can base the labelling functions on them.
    </div>


In [91]:
#Importing good and bad words & preparing for labelling function 

In [92]:
#POSITIVE WORDS 
#positive words from --> DON't DELETE! NEED TO CITE PROPERLY http://ptrckprry.com/course/ssd/data/positive-words.txt
positive_word = pd.read_csv('/project/positive_words.csv')

#sample 500 words 
positive_word = positive_word.sample(500)

#convert it into a list 
positive_word= positive_word['a+'].tolist()



#NEGATIVE WORDS 
#negative words from --> HONG?? 
negative_word = pd.read_csv('/project/negative_words.csv')

#sample 500 words 
negative_word = negative_word.sample(500)

#convert it into a list 
negative_word= negative_word['2-faces'].tolist()


In [144]:
@labeling_function()
def negative(x): 
    for word in negative_word:
        if word in str(x).lower():
            return NOTFRESH 
    return ABSTAIN 



@labeling_function()
def positive(x):
    for word in positive_word:
        if word in str(x).lower():
            return FRESH 
    return ABSTAIN 

In [145]:
lfs = [negative]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [146]:
coverage_negative = (sample_L != ABSTAIN).mean(axis=0)
print("negative words coverage:{:.1%}".format(coverage_negative[0]))

negative words coverage:20.2%


In [147]:
lfs = [positive]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [148]:
coverage_positive = (sample_L != ABSTAIN).mean(axis=0)
print("positive words coverage:{:.1%}".format(coverage_positive[0]))

positive words coverage:100.0%


### Labelling Function

## 2. Punctuation occurrences

In [99]:
# Turn review column into Series
development_split_fresh_series = pd.Series(development_split_fresh.Review)
development_split_rotten_series = pd.Series(development_split_rotten.Review)

In [100]:
# Positive reviews
# Split reviews into word
fresh_split = pd.Series(development_split_fresh_series.str.split(expand=True).stack())
fresh_words = [i for i in fresh_split]

# Split words into characters
def split_str():
    return [list(ch) for ch in fresh_words]
fresh_split_words = pd.Series(split_str())

In [101]:
# Negative reviews
# Split reviews into word
rotten_split = pd.Series(development_split_rotten_series.str.split(expand=True).stack())
rotten_words = [i for i in rotten_split]

# Split words into characters
def split_str():
    return [list(ch) for ch in rotten_words]
rotten_split_words = pd.Series(split_str())

In [102]:
# Turn into a flattened list
fresh_flattened_list = [y for x in fresh_split_words for y in x]
rotten_flattened_list = [y for x in rotten_split_words for y in x]

# Count the occurancy of each character
# Positive reviews
fresh_split_characters = pd.Series(fresh_flattened_list).value_counts()
fresh_split_characters = pd.DataFrame(fresh_split_characters).reset_index()

# Negative reviews
rotten_split_characters = pd.Series(rotten_flattened_list).value_counts()
rotten_split_characters = pd.DataFrame(rotten_split_characters).reset_index()

### A. Question mark occurrences

In [103]:
# Count the # of occurance of '?' in fresh reviews
fresh_split_characters[fresh_split_characters['index'] == '?']

,index,0
64,?,17


In [104]:
# Count the # of occurance of '?' in rotten reviews
rotten_split_characters[rotten_split_characters['index'] == '?']

,index,0
53,?,26


In [105]:
list_with_question_mark = []
for review in development_split_rotten.Review:
    if '?' in review:
        list_with_question_mark.append(review)
        
print (list_with_question_mark[:3])

["Critters Attack! might entertain the biggest B-movie horror fans, but with a script this vapid, what's the point?", "Smug? Damn, this is a self-satisfied, arrogant excuse for a film, in which the red-herringness is piled on with outrageous aplomb till there can be no doubt that Twohy is daring you to guess what's up and hoping you're not up to it, or not caring if you a", 'Preposterous? Yes. Ridiculous? That, too.']


In [149]:
@labeling_function()
def question_mark(x):
    return NOTFRESH if '?' in str(x).lower() else ABSTAIN

In [150]:
lfs = [question_mark]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [151]:
coverage_question_mark = (sample_L != ABSTAIN).mean(axis=0)
print("question mark coverage:{:.1%}".format(coverage_question_mark[0]))

question mark coverage:1.8%


### B. Exclaimation mark occurrences

In [109]:
# Count the # of occurance of '!' in fresh reviews
fresh_split_characters[fresh_split_characters['index'] == '!']

,index,0
69,!,9


In [110]:
# Count the # of occurance of '!' in rotten reviews
rotten_split_characters[rotten_split_characters['index'] == '!']

,index,0
68,!,6


In [152]:
@labeling_function()
def exclamation_mark(x):
    return NOTFRESH if '!' in str(x).lower() else ABSTAIN

In [153]:
lfs = [exclamation_mark]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [154]:
coverage_exclamation_mark = (sample_L != ABSTAIN).mean(axis=0)
print("exclaimation mark coverage:{:.1%}".format(coverage_exclamation_mark[0]))

exclaimation mark coverage:0.7%


## Sensitivity Analysis

In [160]:
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(str(x))
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

In [161]:
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return FRESH if x.polarity > 0.8 else ABSTAIN

In [162]:
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return FRESH if x.subjectivity >= 0.5 else ABSTAIN

In [163]:
lfs = [textblob_polarity, textblob_subjectivity]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)





Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 28.0 failed 1 times, most recent failure: Lost task 1.0 in stage 28.0 (TID 113, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/anaconda/envs/Python3/lib/python3.6/site-packages/pyspark/python/lib/p

In [ ]:
LFAnalysis(L_sample, lfs).lf_summary()

# Classifier